# Pointing Model Utility 

This notebook is intended to help building initial pointing model for the Auxiliary Telescope. 

The user can define a grid of Az/El pointings that will be converted to RA/Dec target commands to the ATPtg. For each pointing the user can center the target and register the pointing throught ATPtg. The data can then be saved and used as input to TPOINT to compute pointing model parameters. 

Initially the process is very interactive. Once the process is well stablished we can turn this into a more automated process.

## Building Az/El pointing grid.

In [1]:
import numpy as np

import asyncio 
import logging
import matplotlib.pyplot as plt

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, AltAz, ICRS, EarthLocation, Angle
from astroquery.vo_conesearch import ConeSearch
from astroquery.simbad import Simbad

import ipywidgets as widgets

%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.CRITICAL)

In [4]:
logging.getLogger("astropy*").setLevel(logging.CRITICAL)

## Generate az/el pointing grid

Next step is setting up the parameters for the run. Currently we use a simple `NxN` pointing scheme in both azimuth and altitude. This will likely need to be improved as regions with lower elevation may need a better cover. 


In [ ]:
n_az = 4  # Maximum number of pointings in azimuth
n_alt = 5  # Number of pointings in altitude
mag_limit = 9  # Magnitude limit


In [ ]:
alt_min = Angle(20.*u.deg)
alt_max = Angle(70.*u.deg)
d_alt = (alt_max-alt_min) / n_alt

In [ ]:
az_min = Angle(0.*u.deg)
az_max = Angle(360.*u.deg)

In [ ]:
alt_grid = np.linspace(alt_min, alt_max, n_alt)

In [ ]:
alt_grid = np.logspace(
    np.log10(alt_min.value),
    np.log10(alt_max.value),
    n_alt
)

In [ ]:
alt_grid

Building the Azimuth grid. 

For each altitude it will select a quasi-uniform number of values in azimuth.

In [ ]:
az_grid = np.zeros((n_alt, n_az))
d_az = (az_max-az_min) / n_az

for i in range(n_alt):
    az_grid[i] += np.linspace(az_min.value, az_max.value, n_az+1)[:-1] + d_az.value/n_alt*i


In [ ]:
az_grid

At this point altitude is stored in a 1-d array and azimuth is stored in a matrix. I'll flatten these arrays so they are both 1-d with a one to one mapping.

I'm also preparing the order of the pointings here so we will go over from lower to higher altitutes instead of doing a constant altitude sweap. 

In [ ]:
az_pt_grid = np.zeros(n_alt*n_az)
el_pt_grid = np.zeros(n_alt*n_az)

for i in range(n_az):
    for j in range(n_alt):
        az_pt_grid[i*n_alt + j] = az_grid[j][i]
        el_pt_grid[i*n_alt + j] = alt_grid[j]

In [ ]:
az_pt_grid, el_pt_grid

Make a polar plot of the pointings

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='polar')
ax.set_theta_zero_location('W', offset=0)
ax.set_rlim(90, 0, 1)

c = ax.scatter(np.radians(az_pt_grid), el_pt_grid)
l = ax.plot(np.radians(az_pt_grid), el_pt_grid, ":")
print(len(az_pt_grid))


Now for each Az/El pointing try to find a bright star to point.

In [7]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                                    lat=-30.244728*u.deg,
                                                    height=2663.0*u.m)

In [10]:
customSimbad = Simbad()
customSimbad.add_votable_fields('distance_result', "fluxdata(V)")
customSimbad.TIMEOUT = 30

In [ ]:
def wait_center():
    opt = input("Press <enter> to continue or 't' <enter> to terminate")

    if opt == 't':
        print("Stopping test...")
        return -1
    else:
        return 0
    

In [5]:
def find_target(az, el, mag_limit):
    current_time = Time.now()

    elaz = AltAz(alt=el*u.deg, az=az*u.deg, 
                 obstime=current_time.tai, 
                 location=location)

    radec = elaz.transform_to(ICRS)

    _ra = radec.ra.to_string(u.deg, decimal=True)
    _dec = radec.dec.to_string(u.deg, decimal=True, alwayssign=True)
    r = Angle(2.*u.deg).to_string(u.deg, decimal=True)

    criteria = f"region(circle,ICRS,{_ra} {_dec},{r}d) & Vmag > {mag_limit} & Vmag < {mag_limit+2} & cat = HD"

    try:
        result_table = customSimbad.query_criteria(criteria)
        if result_table is None:
            raise RuntimeError("No result from query.")
    except Exception as e:
        print(f"Error getting target from Simbad. Skipping target {elaz!r}.")
        print(criteria)
        return -1

    result_table.sort('FLUX_V')

    target_radec = SkyCoord(Angle(result_table['RA'][0], unit=u.hour), 
                    Angle(result_table['DEC'][0], unit=u.deg), frame=ICRS)

    t_id = str(result_table['MAIN_ID'][0])
    t_ra = result_table['RA'][0]
    t_dec = result_table['DEC'][0]
    t_mag = result_table['FLUX_V'][0]
    print(f">>>> Point to {t_id:15}: {t_ra} {t_dec} (VMag:{t_mag:5.2f})")

    return 0

In [ ]:
for i in range(len(el_pt_grid)):
    retval = find_target(az_pt_grid[i], el_pt_grid[i], mag_limit=mag_limit)

    if retval == 0:
        if wait_center() < 0:
            print(f"Finished in iteration {i}")
            break

In [24]:
find_target(210, 70, 7.0)

>>>> Point to HD 120144      : 13 48 39.7332 -46 58 25.382 (VMag: 7.12)


0